### Load data

In [1]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings("ignore") 
df = pd.read_csv('ha_data.csv', sep=';')
df['time'] = pd.to_datetime(df['time'])
df.head(5)

,order_id,page_id,product,site_version,time,title,user
0,NaN,3.0,company,mobile,2017-02-09 20:24:04,banner_show,user_0
1,NaN,3699687.0,company,mobile,2017-02-07 10:03:07,banner_show,user_0
2,NaN,14.0,sneakers,mobile,2017-01-29 13:02:23,banner_show,user_1
3,NaN,10362176.0,company,mobile,2017-04-12 15:39:19,banner_show,user_1
4,NaN,14.0,sneakers,mobile,2017-01-29 13:04:42,banner_click,user_1


In [2]:
print(df['time'].max() - df['time'].min())

150 days 23:59:55


### Split the users, who have shopped through the desktop and mobile versions

In [3]:
df_desktop = df[df["site_version"] == "desktop"]
df_mobile = df[df["site_version"] == "mobile"]
df_desktop.drop(['site_version'], axis=1, inplace=True)
df_mobile.drop(['site_version'], axis=1, inplace=True)

### Add some informative columns, refactor the data

In [4]:
df_desktop['clicked_company'] = np.where((df_desktop['title'] == 'banner_click') & (df_desktop['product'] == 'company'), 1, 0)
df_desktop['clicked_company'] = df_desktop.groupby('user')['clicked_company'].transform(max)
df_mobile['clicked_company'] = np.where((df_mobile['title'] == 'banner_click') & (df_mobile['product'] == 'company'), 1, 0)
df_mobile['clicked_company'] = df_mobile.groupby('user')['clicked_company'].transform(max)

df_desktop['ordered'] = np.where((df_desktop['order_id'].isna()), 0, 1)
df_desktop['ordered'] = df_desktop.groupby('user')['ordered'].transform(max)
df_desktop['clicked_nordered'] = np.where(((df_desktop['ordered'] == 0) & (df_desktop['title'] == 'banner_click')), 1, 0)
df_desktop['clicked_nordered'] = df_desktop.groupby('user')['clicked_nordered'].transform(max)

df_mobile['ordered'] = np.where((df_mobile['order_id'].isna()), 0, 1)
df_mobile['ordered'] = df_mobile.groupby('user')['ordered'].transform(max)
df_mobile['clicked_nordered'] = np.where(((df_mobile['ordered'] == 0) & (df_mobile['title'] == 'banner_click')), 1, 0)
df_mobile['clicked_nordered'] = df_mobile.groupby('user')['clicked_nordered'].transform(max)

df_desktop_company_click = df_desktop[df_desktop['clicked_company'] == 1]
df_desktop_other_users = df_desktop[df_desktop['clicked_company'] == 0]
df_mobile_company_click = df_mobile[df_mobile['clicked_company'] == 1]
df_mobile_other_users = df_mobile[df_mobile['clicked_company'] == 0]

df.drop(['page_id', 'order_id'], axis=1, inplace=True)

### Convertion with target action of order

In [5]:
pt_desktop_ordered = df_desktop.pivot_table(index=['user'], values='ordered', aggfunc='sum')
valid_desktop_user_ordered = pt_desktop_ordered[pt_desktop_ordered['ordered'] > 0].index
count_desktop_orders = len(pt_desktop_ordered.loc[valid_desktop_user_ordered])

pt_mobile_ordered = df_mobile.pivot_table(index=['user'], values='ordered', aggfunc='sum')
valid_mobile_user_ordered = pt_mobile_ordered[pt_mobile_ordered['ordered'] > 0].index
count_mobile_orders = len(pt_mobile_ordered.loc[valid_mobile_user_ordered])

print("Desktop Conversion with a target action of order: {:.2f}%".format(count_desktop_orders / len(df_desktop) * 100))
print("Mobile Conversion with a target action of order: {:.2f}%".format(count_mobile_orders / len(df_mobile) * 100))
print("Desktop orders : {:.2f}".format(df_desktop['user'].nunique() * np.round(count_desktop_orders / len(df_desktop), 2)))
print("Mobile orders : {:.2f}".format(df_mobile['user'].nunique() * np.round(count_mobile_orders / len(df_mobile), 2)))

Desktop Conversion with a target action of order: 5.26%
Mobile Conversion with a target action of order: 1.85%
Desktop orders : 71238.60
Mobile orders : 57143.78


### Convertion with target action of clicking, but not buying

In [6]:
pt_desktop_clicked_nordered = df_desktop.pivot_table(index=['user'], values='clicked_nordered', aggfunc='sum')
pt_desktop_clicked_lordered = df_desktop.pivot_table(index=['user', 'product'], columns='title', values='time', aggfunc='min')

#clicked on the banner & not ordered
valid_user_desktop_cno = (pt_desktop_clicked_nordered[pt_desktop_clicked_nordered['clicked_nordered'] > 0].index.get_level_values('user')).to_list()
#clicked on the banner only after they have already made an order
valid_user_desktop_clo = (pt_desktop_clicked_lordered[pt_desktop_clicked_lordered['banner_click'] > pt_desktop_clicked_lordered['order']].index.get_level_values('user')).to_list()

union_desktop = list(dict.fromkeys(valid_user_desktop_cno + valid_user_desktop_clo))

pt_mobile_clicked_nordered = df_mobile.pivot_table(index=['user'], values='clicked_nordered', aggfunc='sum')
pt_mobile_clicked_lordered = df_mobile.pivot_table(index=['user', 'product'], columns='title', values='time', aggfunc='min')

valid_user_mobile_cno = (pt_mobile_clicked_nordered[pt_mobile_clicked_nordered['clicked_nordered'] > 0].index.get_level_values('user')).to_list()
valid_user_mobile_clo = (pt_mobile_clicked_lordered[pt_mobile_clicked_lordered['banner_click'] > pt_mobile_clicked_lordered['order']].index.get_level_values('user')).to_list()

union_mobile = list(dict.fromkeys(valid_user_mobile_cno + valid_user_mobile_clo))

### Convertion with target action of order after the banner click

In [7]:
# those with the banner clicked before the order, haven't clicked on a company banner - desktop
pt_desktop_nclicked = df_desktop_other_users.pivot_table(index=['user', 'product'], columns='title', values='time', aggfunc='min')
valid_user_desktop_nclicked = pt_desktop_nclicked[pt_desktop_nclicked['banner_click'] < pt_desktop_nclicked['order']].index
num_partitions_desktop_nclicked = len(pt_desktop_nclicked.loc[valid_user_desktop_nclicked])

# those with the banner clicked before the order, have clicked on a company banner - desktop
pt_desktop_clicked = df_desktop_company_click.pivot_table(index=['user'], columns='title', values='time', aggfunc='min')
valid_user_desktop_clicked = pt_desktop_clicked[pt_desktop_clicked['banner_click'] < pt_desktop_clicked['order']].index
num_partitions_desktop_nclicked += len(pt_desktop_clicked.loc[valid_user_desktop_clicked])

# those with the banner clicked before the order, haven't clicked on a company banner - mobile
pt_mobile_nclicked = df_mobile_other_users.pivot_table(index=['user', 'product'], columns='title', values='time', aggfunc='min')
valid_user_mobile_nclicked = pt_mobile_nclicked[pt_mobile_nclicked['banner_click'] < pt_mobile_nclicked['order']].index
num_partitions_mobile_nclicked = len(pt_mobile_nclicked.loc[valid_user_mobile_nclicked])

# those with the banner clicked before the order, have clicked on a company banner - mobile
pt_mobile_clicked = df_mobile_company_click.pivot_table(index=['user'], columns='title', values='time', aggfunc='min')
valid_user_mobile_clicked = pt_mobile_clicked[pt_mobile_clicked['banner_click'] < pt_mobile_clicked['order']].index
num_partitions_mobile_nclicked += len(pt_mobile_clicked.loc[valid_user_mobile_clicked])

print("Desktop Conversion with a target action of clicking: {:.4f}%".format(len(union_desktop)/len(df_desktop) * 100 + num_partitions_mobile_nclicked/len(df_mobile) * 100))
print("Mobile Conversion with a target action of clicking: {:.4f}%".format(len(union_mobile)/len(df_mobile) * 100 + num_partitions_desktop_nclicked/len(df_desktop) * 100))
print("Desktop Conversion with a target action of clicking & buying: {:.4f}%".format(num_partitions_mobile_nclicked/len(df_mobile) * 100))
print("Mobile Conversion with a target action of clicking & buying: {:.4f}%".format(num_partitions_desktop_nclicked/len(df_desktop) * 100))

Desktop Conversion with a target action of clicking: 3.7911%
Mobile Conversion with a target action of clicking: 9.6231%
Desktop Conversion with a target action of clicking & buying: 0.2776%
Mobile Conversion with a target action of clicking & buying: 0.4017%


### Example of one user partition

In [8]:
df_desktop[df_desktop['user'] == 'user_1000698']

,order_id,page_id,product,time,title,user,clicked_company,ordered,clicked_nordered
2286106,NaN,5253780.0,accessories,2017-01-12 17:48:24,banner_show,user_1000698,0,0,1
2286107,NaN,8711274.0,clothes,2017-03-22 02:22:09,banner_show,user_1000698,0,0,1
2286108,NaN,15762622.0,clothes,2017-04-07 07:09:42,banner_show,user_1000698,0,0,1
2286109,NaN,15762622.0,clothes,2017-04-07 07:11:10,banner_click,user_1000698,0,0,1


In [9]:
mobile = df_mobile['user'].nunique() * np.round(count_mobile_orders / len(df_mobile), 3)
desktop = df_desktop['user'].nunique() * np.round(count_desktop_orders / len(df_desktop), 3)
print("We risk to loose: {:.0f} mobile clients".format(mobile))
print("We risk to loose: {:.0f} desktop clients".format(desktop))
print("In summary the lower bound would be: {:.0f} clients".format(desktop+mobile))

We risk to loose: 51429 mobile clients
We risk to loose: 75513 desktop clients
In summary the lower bound would be: 126942 clients


In [10]:
a = df_desktop['user'].nunique()
b = df_mobile['user'].nunique()
print(f'Users in desktop version {a}')
print(f'Users in mobile version {b}')

Users in desktop version 1424772
Users in mobile version 2857189
